# 文本生成案例——LSTM

In [1]:
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import sklearn
import pandas as pd
import os
import sys
import time
import tensorflow as tf

from tensorflow import keras

print(tf.__version__)
print(sys.version_info)
for module in mpl, np, pd, sklearn, tf, keras:
    print(module.__name__, module.__version__)

2.0.0
sys.version_info(major=3, minor=7, micro=7, releaselevel='final', serial=0)
matplotlib 3.2.2
numpy 1.18.5
pandas 1.0.5
sklearn 0.21.2
tensorflow 2.0.0
tensorflow_core.keras 2.2.4-tf


In [2]:
# https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt
input_filepath = "./data/shakespeare.txt"
text = open(input_filepath, 'r').read()

print(len(text))
print(text[0:100])

1115393
First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You


In [3]:
# 1. generate vocab
# 2. build mapping char->id
# 3. data -> id_data
# 4. abcd -> bcd<eos>

# 生成词汇列表
vocab = sorted(set(text))
print(len(vocab))
print(vocab)

65
['\n', ' ', '!', '$', '&', "'", ',', '-', '.', '3', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']


In [4]:
# 生成{单词:id}的字典
char2idx = {char:idx for idx, char in enumerate(vocab)}
print(char2idx)

{'\n': 0, ' ': 1, '!': 2, '$': 3, '&': 4, "'": 5, ',': 6, '-': 7, '.': 8, '3': 9, ':': 10, ';': 11, '?': 12, 'A': 13, 'B': 14, 'C': 15, 'D': 16, 'E': 17, 'F': 18, 'G': 19, 'H': 20, 'I': 21, 'J': 22, 'K': 23, 'L': 24, 'M': 25, 'N': 26, 'O': 27, 'P': 28, 'Q': 29, 'R': 30, 'S': 31, 'T': 32, 'U': 33, 'V': 34, 'W': 35, 'X': 36, 'Y': 37, 'Z': 38, 'a': 39, 'b': 40, 'c': 41, 'd': 42, 'e': 43, 'f': 44, 'g': 45, 'h': 46, 'i': 47, 'j': 48, 'k': 49, 'l': 50, 'm': 51, 'n': 52, 'o': 53, 'p': 54, 'q': 55, 'r': 56, 's': 57, 't': 58, 'u': 59, 'v': 60, 'w': 61, 'x': 62, 'y': 63, 'z': 64}


In [5]:

idx2char = np.array(vocab)
print(idx2char)

['\n' ' ' '!' '$' '&' "'" ',' '-' '.' '3' ':' ';' '?' 'A' 'B' 'C' 'D' 'E'
 'F' 'G' 'H' 'I' 'J' 'K' 'L' 'M' 'N' 'O' 'P' 'Q' 'R' 'S' 'T' 'U' 'V' 'W'
 'X' 'Y' 'Z' 'a' 'b' 'c' 'd' 'e' 'f' 'g' 'h' 'i' 'j' 'k' 'l' 'm' 'n' 'o'
 'p' 'q' 'r' 's' 't' 'u' 'v' 'w' 'x' 'y' 'z']


In [6]:
# 生成莎士比亚文本中每个单词对应的id的ndarray
text_as_int = np.array([char2idx[c] for c in text])
print(text_as_int[0:10])
print(text[0:10])

# F对应id是18，i对应的id是47...

[18 47 56 57 58  1 15 47 58 47]
First Citi


In [7]:
def split_input_target(id_text):
    """
    把输入的文本拆分成特征值和目标值
    abcde -> abcd, bcde
    a的目标值是b
    b的目标值是c
    ...
    """
    return id_text[0:-1], id_text[1:]

In [8]:
# 单词id的dataset
char_dataset =tf.data.Dataset.from_tensor_slices(text_as_int)
for ch_id in char_dataset.take(2):
    print(ch_id, idx2char[ch_id.numpy()])

tf.Tensor(18, shape=(), dtype=int32) F
tf.Tensor(47, shape=(), dtype=int32) i


In [9]:
repr("".join(idx2char[[18,47,56,57,58,1]]))

"'First '"

In [10]:

seq_length = 100

# 转变成句子的dataset，长度+1：因为split_input_target函数输出的特征值和目标值会减1，所以+1
# drop_remainder= True：最后一个batch不够长了就丢弃
seq_dataset = char_dataset.batch(seq_length + 1,
                                 drop_remainder = True)

for seq_id in seq_dataset.take(2):
    print(seq_id)
    print(seq_id.numpy())
    print(repr(''.join(idx2char[seq_id.numpy()])))

tf.Tensor(
[18 47 56 57 58  1 15 47 58 47 64 43 52 10  0 14 43 44 53 56 43  1 61 43
  1 54 56 53 41 43 43 42  1 39 52 63  1 44 59 56 58 46 43 56  6  1 46 43
 39 56  1 51 43  1 57 54 43 39 49  8  0  0 13 50 50 10  0 31 54 43 39 49
  6  1 57 54 43 39 49  8  0  0 18 47 56 57 58  1 15 47 58 47 64 43 52 10
  0 37 53 59  1], shape=(101,), dtype=int32)
[18 47 56 57 58  1 15 47 58 47 64 43 52 10  0 14 43 44 53 56 43  1 61 43
  1 54 56 53 41 43 43 42  1 39 52 63  1 44 59 56 58 46 43 56  6  1 46 43
 39 56  1 51 43  1 57 54 43 39 49  8  0  0 13 50 50 10  0 31 54 43 39 49
  6  1 57 54 43 39 49  8  0  0 18 47 56 57 58  1 15 47 58 47 64 43 52 10
  0 37 53 59  1]
'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '
tf.Tensor(
[39 56 43  1 39 50 50  1 56 43 57 53 50 60 43 42  1 56 39 58 46 43 56  1
 58 53  1 42 47 43  1 58 46 39 52  1 58 53  1 44 39 51 47 57 46 12  0  0
 13 50 50 10  0 30 43 57 53 50 60 43 42  8  1 56 43 57 53 50 60 43 42  8
  

In [11]:
# 通过split_input_target函数生成包含特征值和目标值的dataset
seq_dataset = seq_dataset.map(split_input_target)
for item_input, item_output in seq_dataset.take(2):
    print(item_input.numpy())
    print(item_output.numpy())

[18 47 56 57 58  1 15 47 58 47 64 43 52 10  0 14 43 44 53 56 43  1 61 43
  1 54 56 53 41 43 43 42  1 39 52 63  1 44 59 56 58 46 43 56  6  1 46 43
 39 56  1 51 43  1 57 54 43 39 49  8  0  0 13 50 50 10  0 31 54 43 39 49
  6  1 57 54 43 39 49  8  0  0 18 47 56 57 58  1 15 47 58 47 64 43 52 10
  0 37 53 59]
[47 56 57 58  1 15 47 58 47 64 43 52 10  0 14 43 44 53 56 43  1 61 43  1
 54 56 53 41 43 43 42  1 39 52 63  1 44 59 56 58 46 43 56  6  1 46 43 39
 56  1 51 43  1 57 54 43 39 49  8  0  0 13 50 50 10  0 31 54 43 39 49  6
  1 57 54 43 39 49  8  0  0 18 47 56 57 58  1 15 47 58 47 64 43 52 10  0
 37 53 59  1]
[39 56 43  1 39 50 50  1 56 43 57 53 50 60 43 42  1 56 39 58 46 43 56  1
 58 53  1 42 47 43  1 58 46 39 52  1 58 53  1 44 39 51 47 57 46 12  0  0
 13 50 50 10  0 30 43 57 53 50 60 43 42  8  1 56 43 57 53 50 60 43 42  8
  0  0 18 47 56 57 58  1 15 47 58 47 64 43 52 10  0 18 47 56 57 58  6  1
 63 53 59  1]
[56 43  1 39 50 50  1 56 43 57 53 50 60 43 42  1 56 39 58 46 43 56  1 58
 53  1 42

In [12]:
batch_size = 64
buffer_size = 10000

# 打乱数据集
seq_dataset = seq_dataset.shuffle(buffer_size).batch(
    batch_size, drop_remainder=True)

In [13]:
# 定义模型
vocab_size = len(vocab)
embedding_dim = 256
rnn_units = 1024

def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
    model = keras.models.Sequential([
        keras.layers.Embedding(vocab_size, embedding_dim,
                               batch_input_shape = [batch_size, None]),
        keras.layers.LSTM(units = rnn_units,
                               stateful = True,
                               recurrent_initializer = 'glorot_uniform',
                               return_sequences = True),
        keras.layers.Dense(vocab_size),
    ])
    return model

model = build_model(
    vocab_size = vocab_size,
    embedding_dim = embedding_dim,
    rnn_units = rnn_units,
    batch_size = batch_size)

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (64, None, 256)           16640     
_________________________________________________________________
lstm (LSTM)                  (64, None, 1024)          5246976   
_________________________________________________________________
dense (Dense)                (64, None, 65)            66625     
Total params: 5,330,241
Trainable params: 5,330,241
Non-trainable params: 0
_________________________________________________________________


In [14]:
for input_example_batch, target_example_batch in seq_dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape)
    print(example_batch_predictions)

# 生成概率分布，对应65个字符的概率分布

(64, 100, 65)
tf.Tensor(
[[[-1.1876326e-03  9.7733317e-04 -7.3851668e-03 ...  3.2300442e-03
   -3.3626442e-03 -2.2406557e-03]
  [ 7.7993209e-03  1.5103130e-03 -6.3130804e-03 ...  8.2435869e-03
    2.1060237e-03  5.1497063e-04]
  [ 9.4396202e-03  8.8268472e-04 -1.1276608e-02 ...  3.1948793e-03
    3.7252796e-03  1.4014239e-03]
  ...
  [-6.2655951e-03 -6.4251781e-03 -8.9515503e-03 ... -6.9505018e-03
    1.2059066e-02  1.0622805e-02]
  [-4.7497796e-03 -4.4976412e-03 -1.0173734e-02 ... -1.3298463e-03
    7.8918189e-03  9.5540788e-03]
  [-3.8118456e-03 -2.7011931e-03 -1.5954766e-02 ...  2.4028164e-03
    2.1283012e-03  5.5212700e-03]]

 [[-8.8418939e-04  2.6744162e-03  5.3327610e-03 ... -8.5929455e-04
    5.9714238e-04 -3.5738520e-04]
  [-1.6508397e-03  5.1857885e-03  6.2998286e-03 ... -8.3844457e-03
    4.5520952e-05  7.3345681e-04]
  [-3.4527716e-03  2.6132623e-03  2.2972221e-03 ... -9.7772796e-03
    4.3315212e-03  3.4748528e-03]
  ...
  [-1.1705618e-02 -1.7461402e-04 -6.9204774e-03 ... 

In [15]:
# 随机采样 random sampling.
# greedy, random.
sample_indices = tf.random.categorical(
    logits = example_batch_predictions[0], num_samples = 1)
print(sample_indices)
# (100, 65) -> (100, 1)

# 把1的维度消除
sample_indices = tf.squeeze(sample_indices, axis = -1)
print(sample_indices)

tf.Tensor(
[[23]
 [53]
 [ 7]
 [ 0]
 [47]
 [54]
 [38]
 [ 0]
 [39]
 [17]
 [54]
 [61]
 [13]
 [23]
 [27]
 [44]
 [30]
 [47]
 [ 7]
 [ 7]
 [47]
 [41]
 [24]
 [ 2]
 [28]
 [43]
 [12]
 [63]
 [14]
 [35]
 [31]
 [51]
 [18]
 [24]
 [26]
 [26]
 [21]
 [42]
 [55]
 [47]
 [23]
 [ 2]
 [41]
 [ 0]
 [ 3]
 [45]
 [29]
 [14]
 [58]
 [13]
 [ 3]
 [28]
 [34]
 [28]
 [ 8]
 [ 5]
 [30]
 [ 1]
 [55]
 [64]
 [48]
 [44]
 [ 9]
 [47]
 [ 7]
 [26]
 [34]
 [36]
 [47]
 [15]
 [ 7]
 [51]
 [43]
 [ 8]
 [ 4]
 [58]
 [43]
 [44]
 [10]
 [60]
 [16]
 [20]
 [42]
 [32]
 [23]
 [ 6]
 [42]
 [22]
 [61]
 [24]
 [32]
 [50]
 [47]
 [60]
 [53]
 [ 8]
 [58]
 [50]
 [ 2]
 [19]], shape=(100, 1), dtype=int64)
tf.Tensor(
[23 53  7  0 47 54 38  0 39 17 54 61 13 23 27 44 30 47  7  7 47 41 24  2
 28 43 12 63 14 35 31 51 18 24 26 26 21 42 55 47 23  2 41  0  3 45 29 14
 58 13  3 28 34 28  8  5 30  1 55 64 48 44  9 47  7 26 34 36 47 15  7 51
 43  8  4 58 43 44 10 60 16 20 42 32 23  6 42 22 61 24 32 50 47 60 53  8
 58 50  2 19], shape=(100,), dtype=int64)


In [16]:
print("Input: ", repr("".join(idx2char[input_example_batch[0]])))
print()
print("Output: ", repr("".join(idx2char[target_example_batch[0]])))
print()
print("Predictions: ", repr("".join(idx2char[sample_indices])))

Input:  "iled keys off that hung in\nchains: no hearing, no feeling, but my sir's song,\nand admiring the nothi"

Output:  "led keys off that hung in\nchains: no hearing, no feeling, but my sir's song,\nand admiring the nothin"

Predictions:  "Ko-\nipZ\naEpwAKOfRi--icL!Pe?yBWSmFLNNIdqiK!c\n$gQBtA$PVP.'R qzjf3i-NVXiC-me.&tef:vDHdTK,dJwLTlivo.tl!G"


In [17]:
def loss(labels, logits):
    """
    自定义损失函数
    """
    return keras.losses.sparse_categorical_crossentropy(
        labels, logits, from_logits=True)

model.compile(optimizer = 'adam', loss = loss)
example_loss = loss(target_example_batch, example_batch_predictions)
print(example_loss.shape)
print(example_loss.numpy().mean())

(64, 100)
4.174142


In [18]:
output_dir = "./text_generation_checkpoints"
if not os.path.exists(output_dir):
    os.mkdir(output_dir)
checkpoint_prefix = os.path.join(output_dir, 'ckpt_{epoch}')
checkpoint_callback = keras.callbacks.ModelCheckpoint(
    filepath = checkpoint_prefix,
    save_weights_only = True)

epochs = 100
history = model.fit(seq_dataset, epochs = epochs,
                    callbacks = [checkpoint_callback])

Epoch 1/100
172/172 [==============================] - 36s 209ms/step - loss: 2.5672
Epoch 2/100
172/172 [==============================] - 34s 195ms/step - loss: 1.8660
Epoch 3/100
172/172 [==============================] - 34s 195ms/step - loss: 1.6242
Epoch 4/100
172/172 [==============================] - 33s 195ms/step - loss: 1.4957
Epoch 5/100
172/172 [==============================] - 34s 196ms/step - loss: 1.4195
Epoch 6/100
172/172 [==============================] - 34s 197ms/step - loss: 1.3643
Epoch 7/100
172/172 [==============================] - 34s 196ms/step - loss: 1.3209
Epoch 8/100
172/172 [==============================] - 34s 196ms/step - loss: 1.2833
Epoch 9/100
172/172 [==============================] - 34s 195ms/step - loss: 1.2471
Epoch 10/100
172/172 [==============================] - 34s 196ms/step - loss: 1.2130
Epoch 11/100
172/172 [==============================] - 34s 196ms/step - loss: 1.1781
Epoch 12/100
172/172 [==============================] - 34s 196

172/172 [==============================] - 34s 196ms/step - loss: 0.3906
Epoch 95/100
172/172 [==============================] - 34s 197ms/step - loss: 0.3906
Epoch 96/100
172/172 [==============================] - 34s 197ms/step - loss: 0.3897
Epoch 97/100
172/172 [==============================] - 34s 196ms/step - loss: 0.3896
Epoch 98/100
172/172 [==============================] - 34s 196ms/step - loss: 0.3890
Epoch 99/100
172/172 [==============================] - 34s 196ms/step - loss: 0.39070s - loss: 0.
Epoch 100/100
172/172 [==============================] - 34s 196ms/step - loss: 0.3909


In [19]:
# 查看最新保存的模型
tf.train.latest_checkpoint(output_dir)

'./text_generation_checkpoints\\ckpt_100'

In [20]:
# 载入模型
model2 = build_model(vocab_size,
                     embedding_dim,
                     rnn_units,
                     batch_size = 1)

# 从模型中载入权重参数
model2.load_weights(tf.train.latest_checkpoint(output_dir))
model2.build(tf.TensorShape([1, None]))
# start ch sequence A,
# A -> model -> b
# A.append(b) -> B
# B(Ab) -> model -> c
# B.append(c) -> C
# C(Abc) -> model -> ...
model2.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (1, None, 256)            16640     
_________________________________________________________________
lstm_1 (LSTM)                (1, None, 1024)           5246976   
_________________________________________________________________
dense_1 (Dense)              (1, None, 65)             66625     
Total params: 5,330,241
Trainable params: 5,330,241
Non-trainable params: 0
_________________________________________________________________


In [21]:
def generate_text(model, start_string, num_generate = 1000):
    input_eval = [char2idx[ch] for ch in start_string]
    input_eval = tf.expand_dims(input_eval, 0)
    
    text_generated = []
    model.reset_states()
    
    # temperature > 1, random
    # temperature < 1, greedy 
    temperature = 2
    
    for _ in range(num_generate):
        # 1. model inference -> predictions
        # 2. sample -> ch -> text_generated.
        # 3. update input_eval
        
        # predictions : [batch_size, input_eval_len, vocab_size]
        predictions = model(input_eval)
        # predictions: logits -> softmax -> prob
        # softmax: e^xi 
        # eg: 4,2 e^4/(e^4 + e^2) = 0.88, e^2 / (e^4 + e^2) = 0.12
        # eg: 2,1 e^2/(e^2 + e) = 0.73, e / (e^2 + e) = 0.27
        predictions = predictions / temperature
        # predictions : [input_eval_len, vocab_size]
        predictions = tf.squeeze(predictions, 0)
        # predicted_ids: [input_eval_len, 1]
        # a b c -> b c d
        predicted_id = tf.random.categorical(
            predictions, num_samples = 1)[-1, 0].numpy()
        text_generated.append(idx2char[predicted_id])
        # s, x -> rnn -> s', y
        input_eval = tf.expand_dims([predicted_id], 0)
    return start_string + ''.join(text_generated)

new_text = generate_text(model2, "All: ")
print(new_text)

All: oy A side but y,
Whipt his most roxes of him! We'll to yon
in yout paith.:

ROMEO:
Pardon is you content til use you owing any thing we sueds to ass, or light
My e sole,
JAMILLO;
Besides, the Lady Vanhon the point acaid!
Down with me Duke of Clambertl.

POLIXENES:
Noble Pause. Can suddenly meant?
What cannot tempe; you are
never cast by, as Volsce Marcar thy joinfut queenus asuke
Had I for Nap, s amam.
Breasts! CORrule to me!
Deap tickess
Plantas art s! live can I fill myself a
Clad right sed thee run away;
Abud Aufidius loves me;
Yet cliff, you n soldier. Shall IWinculeQudiers nor.

DORCAS:
If shadewnix
Bianca
knowing
will I witnessence.
But first Servant:
My life, if York; but ifis I weep?
I would I'll cursu gueedlf;
Upon his part there; yet KING HENRY VI

RICHARD:
Nay, dood? Nam, then muracley? I have arrived at him sende soverended toads! is tro
As thou caitiff! jest she breaking lieutenant, stay with his
friar tellow.

FLORIZEL:
An't plain, ga awake's one
weeping fles farther